## 决策树的构造

In [1]:
from math import log
import numpy as np
import matplotlib.pyplot as plt
import operator

import init

优点：计算复杂度不高，输出结果易理解，对中间值缺失不敏感，可处理不相关特征数据

缺点：可能产生过度匹配问题

适用数据：数值型，标称型

### 信息增益

在划分数据集前后信息发生的变化称为`信息增益`。

计算每个特征值划分数据集获得的信息增益，信息增益最高的特征就是最好的选择。

集合信息的度量方式称为`香农熵`或简称`熵`。

如果待分类的事物可能划分在多个分类中，则符号 $x_i$ 的信息定义为 $$l(x_i) = - \log_2{p(x_i)}$$ 其中 $p(x_i)$ 是选择该分类的概率。

为了计算熵，我们需要计算所有类别所有可能值包含的信息期望值 $$H = -\Sigma^n_{i=1}{p(x_i)\log_2{p(x_i)}}$$ 其中 $n$ 是分类的数目。

In [2]:
def calcShannonEnt(dataSet):
    numEntries = len(dataSet)
    labelCounts = {}
    for featVec in dataSet:
        currentLabel = featVec[-1]
        if currentLabel not in labelCounts.keys():
            labelCounts[currentLabel] = 0
        labelCounts[currentLabel] += 1
    shannoEnt = 0.0
    for key in labelCounts:
        prob = float(labelCounts[key]) / numEntries
        shannoEnt -= prob * log(prob, 2)
    return shannoEnt

In [3]:
def createDataSet():
    # 最后一列是实例的类别标签
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing', 'flippers']
    return dataSet, labels

In [4]:
myDat, labels = createDataSet()

In [5]:
calcShannonEnt(myDat)

0.9709505944546686

熵越高，则混合的数据越多

In [6]:
myDat[0][-1]='maybe'
calcShannonEnt(myDat)

1.3709505944546687

### 划分数据集

In [7]:
def splitDataSet(dataSet, axis, value):
    retDataSet = []
    for featVec in dataSet:
        if featVec[axis] == value:
            reducedFeatVec = featVec[:axis]
            reducedFeatVec.extend(featVec[axis+1:]) # 去除 axis 列
            retDataSet.append(reducedFeatVec)
    return retDataSet

In [8]:
myDat, labels = createDataSet()

In [9]:
myDat

[[1, 1, 'yes'], [1, 1, 'yes'], [1, 0, 'no'], [0, 1, 'no'], [0, 1, 'no']]

In [10]:
splitDataSet(myDat, 0, 1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

In [11]:
splitDataSet(myDat, 0, 0)

[[1, 'no'], [1, 'no']]

In [12]:
def chooseBestFeatureToSplit(dataSet):
    numFeatures = len(dataSet[0]) - 1
    baseEntropy = calcShannonEnt(dataSet)
    bestInfoGain = 0.0; bestFeature = -1
    for i in range(numFeatures):
        featList = [example[i] for example in dataSet]
        uniqueVals = set(featList)
        newEntropy = 0.0
        for value in uniqueVals:
            subDataSet = splitDataSet(dataSet, i, value)
            prob = len(subDataSet) / float(len(dataSet))
            newEntropy += prob * calcShannonEnt(subDataSet)
        infoGain = baseEntropy - newEntropy
        if infoGain > bestInfoGain:
            bestInfoGain = infoGain
            bestFeature = i
    return bestFeature

In [13]:
chooseBestFeatureToSplit(createDataSet()[0])

0

第 0 个特征是最好的用于划分数据集的特征

### 递归构建决策树

In [14]:
def majorityCnt(classList):
    classCount = {}
    for vote in classList:
        if vote not in classCount.keys() : classCount[vote] = 0
        classCount[vote] += 1
    sortedClassCount = sorted(classCount.items(), key=operator.itemgetter(1), reverse=True)
    return sortedClassCount[0][0]

In [15]:
def createTree(dataSet, labels):
    classList = [example[-1] for example in dataSet] # 取类别标签列
    if classList.count(classList[0]) == len(classList): # 类别完全相同
        return classList[0]
    if len(dataSet[0]) == 1:
        return majorityCnt(classList)
    bestFeat = chooseBestFeatureToSplit(dataSet)
    bestFeatLabel = labels[bestFeat]
    myTree = {bestFeatLabel: {}}
    del(labels[bestFeat])
    featValues = [example[bestFeat] for example in dataSet]
    uniqueVals = set(featValues)
    for value in uniqueVals:
        subLabels = labels[:]
        myTree[bestFeatLabel][value] = createTree(splitDataSet(dataSet, bestFeat, value), subLabels)
    return myTree

In [16]:
myDat, labels = createDataSet()
myTree = createTree(myDat, labels)
myTree

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

## Matplotlib 绘制树形图

In [18]:
decisionNode = dict(boxstyle="sawtooth", fc="0.8")
leafNode = dict(boxstyle="round4", fc="0.8")
arrow_args = dict(arrowstyle="<-")

In [19]:
def createPlot():
    fig = plt.figure(1, facecolor='white')
    fig.clf()
    createPlot.ax1 = plt.subplot(111, frameon=False)
    plotNode('决策节点', (0.5, 0.1), (0.1, 0.5), decisionNode)
    plotNode('叶节点', (0.8, 0.1), (0.3, 0.8), leafNode)

In [20]:
def plotNode(nodeTxt, centerPt, parentPt, nodeType):
    createPlot.ax1.annotate(nodeTxt, xy=parentPt, xycoords='axes fraction', xytext=centerPt, textcoords='axes fraction', va="center", ha="center", bbox=nodeType, arrowprops=arrow_args)

In [21]:
createPlot()

AttributeError: 'FancyArrowPatch' object has no property 'arrow_style'